In [1]:
# Librerias

import pandas as pd
import csv
from datetime import datetime
import numpy as np

pacientes = 'diagnostico_3369.csv'

# Parameters: categories for comorbidities and other diseases (DMT2's severities)

severity_cat = ['cardiovascular', 'nefropatia', 'retinopatia', 'enf_periferica', 'acv', 'neuropatia', 'desorden_metab']
comorbidity_cat = ['cancer', 'enf_pulmonar', 'gastrointestinal', 'musculo_esqueletica', 'enf_mental', 'abuso_sustancia']

severity_ciaps = {'cardiovascular': [['K74', 1], ['K75', 2], ['K76', 1], ['K77', 2], ['K78', 2], ['K79', 2], ['K80', 2],
                                     ['K84', 2], ['K91', 1], ['K92', 1], ['K99', 2]],
                  'nefropatia': [['U06', 1], ['U88', 1], ['U99', 2]],
                  'retinopatia': [['F82', 2], ['F83', 1], ['F84', 1]],
                  'enf_periferica': [['S97', 2]],
                  'acv': [['K89', 1], ['K90', 2]],
                  'neuropatia': [['N93', 1], ['N94', 1]],
                  'desorden_metab': [['T87', 2]]}

comorbidity_ciaps = {'cancer': ['A79', 'B72', 'D74', 'F74', 'H75', 'L71', 'K72', 'N74', 'R84', 'R85', 'S77', 'T71',
                                'T93', 'U75', 'U76', 'U77', 'U79', 'W72', 'X75', 'X76', 'X77', 'X81', 'Y77', 'Y78'],
                     'enf_pulmonar': ['R95', 'R96'],
                     'gastrointestinal': ['D84', 'D86', 'D99'],
                     'musculo_esqueletica': ['L84', 'L88', 'L89', 'L90', 'L91', 'L99'],
                     'enf_mental': ['P01', 'P02', 'P27', 'P70', 'P71', 'P72', 'P73', 'P74', 'P76', 'P77'],
                     'abuso_sustancia': ['P15', 'P16', 'P19', 'Z13']}

# Incluye todas las enfermedades en valid_ciaps

valid_ciaps = ['T90']
for cat in severity_ciaps:
    for code in severity_ciaps[cat]:
        ciap = code[0]
        if ciap not in valid_ciaps:
            valid_ciaps.append(ciap)

for cat in comorbidity_ciaps:
    for ciap in comorbidity_ciaps[cat]:
        if ciap not in valid_ciaps:
            valid_ciaps.append(ciap) 
        
patients_log = {}
diagnosis = {}  # {code: [[patient, date], ...]}
cat_severity_before = {}
cat_severity_after = {}
cat_severity_total = {}
cat_comorbidity_before = {}
cat_comorbidity_after = {}
cat_comorbidity_total = {}

severity_index_category = {}
severity_index = {}
comorbidity_index_category = {}
comorbidity_index = {}



In [2]:
# Gets the ids of studied patients
def read_patient_dm():
    tabla = pd.read_csv(pacientes, sep = ";")
    reader = []
    for x in zip(tabla["id_paciente"]):
        if not x[0] in reader:
            reader.append(x[0])   
    
    for patient in reader:
        
        patients_log[patient] = []
        cat_severity_before[patient] = {}
        cat_severity_after[patient] = {}
        cat_severity_total[patient] = {}
        cat_comorbidity_before[patient] = {}
        cat_comorbidity_after[patient] = {}
        cat_comorbidity_total[patient] = {}
        severity_index_category[patient] = {}
        severity_index[patient] = {}
        comorbidity_index_category[patient] = {}
        comorbidity_index[patient] = {}

        for cat in severity_cat:
            cat_severity_before[patient][cat] = 0
            cat_severity_after[patient][cat] = 0
            cat_severity_total[patient][cat] = 0
            severity_index_category[patient][cat] = 0
            severity_index[patient][cat] = 0
        
        for cat in comorbidity_cat:
            cat_comorbidity_before[patient][cat] = 0
            cat_comorbidity_after[patient][cat] = 0
            cat_comorbidity_total[patient][cat] = 0
            comorbidity_index_category[patient][cat] = 0
            comorbidity_index[patient][cat] = 0
      
    print("Number of studied/selected patients: ", len(patients_log))

read_patient_dm()

Number of studied/selected patients:  3369


In [4]:
# Reads diagnosis (for valid ciaps) of selected patients until the max date parameter
def read_diagnosis():
    
    data = pd.read_csv(pacientes, sep=';')
    data['fecha'] = pd.to_datetime(data['fecha'], format="%d-%m-%Y")
    data = data.sort_values(by=['id_paciente', 'fecha'])
    for patient, date, code in zip(data['id_paciente'], data['fecha'], data['ciap']):
        date = date.date()
        if code != 'T90':
            if code not in diagnosis:
                diagnosis[code] = []
            if patient not in [row[0] for row in diagnosis[code]]:
                diagnosis[code].append([patient, date])

read_diagnosis()

In [5]:
def static_patients():
    
    for category in severity_ciaps:
        for code in severity_ciaps[category]:
            ciap = code[0]
            score = code[1]
            
            if ciap not in diagnosis:
                continue
            
            patients_list = diagnosis[ciap]
            
            for patient_tuple in patients_list:
                
                patient_id = patient_tuple[0]
                severity_index_category[patient_id][category] += score
                
                if severity_index[patient_id][category] < 2:
                    severity_index[patient_id][category] = score
                
                if len(patients_log[patient_id]) < 6:
                    continue
                date = patient_tuple[1]
                t90_date = patients_log[patient_id][5]
                if date >= t90_date:
                    cat_severity_after[patient_id][category] += 1
                else:
                    cat_severity_before[patient_id][category] += 1
                cat_severity_total[patient_id][category] += 1

    for patient in severity_index:
        severity_index[patient] = sum(severity_index[patient].values())

    for category in comorbidity_ciaps:
        for ciap in comorbidity_ciaps[category]:
            if ciap not in diagnosis:
                continue
            patients_list = diagnosis[ciap]
            for patient_tuple in patients_list:
                patient_id = patient_tuple[0]
                comorbidity_index_category[patient_id][category] += 1
                if comorbidity_index[patient_id][category] < 1:
                    comorbidity_index[patient_id][category] = 1

                if len(patients_log[patient_id]) < 6:
                    continue
                date = patient_tuple[1]
                t90_date = patients_log[patient_id][5]
                if date >= t90_date:
                    cat_comorbidity_after[patient_id][category] += 1
                else:
                    cat_comorbidity_before[patient_id][category] += 1
                cat_comorbidity_total[patient_id][category] += 1

    for patient in comorbidity_index:
        comorbidity_index[patient] = sum(comorbidity_index[patient].values())

static_patients()

In [6]:
print(comorbidity_index)

{2731: 1, 97400: 1, 32777: 1, 8203: 1, 16397: 2, 16399: 1, 114704: 0, 114706: 1, 28675: 0, 5465: 3, 73752: 2, 65561: 1, 106522: 1, 49181: 0, 65569: 0, 17755: 3, 57382: 0, 24615: 2, 41004: 1, 98351: 0, 57394: 1, 16435: 3, 102295: 0, 73781: 3, 90171: 2, 49214: 0, 8255: 2, 32834: 2, 40971: 2, 32836: 1, 8261: 4, 57415: 1, 8264: 3, 32841: 1, 8267: 2, 114764: 1, 16461: 1, 8278: 2, 123846: 0, 16475: 1, 65628: 1, 73822: 1, 57443: 3, 38193: 2, 110: 2, 114802: 0, 32883: 0, 8310: 2, 82041: 2, 114811: 0, 16511: 1, 90241: 0, 57477: 2, 136: 3, 24715: 5, 16524: 3, 57485: 0, 8335: 3, 24725: 4, 16536: 0, 66927: 0, 57503: 1, 92419: 1, 82085: 0, 1393: 0, 65705: 1, 90285: 2, 185: 2, 98492: 0, 8381: 2, 41151: 1, 192: 2, 114882: 2, 65731: 1, 16580: 2, 5494: 0, 57544: 1, 82121: 2, 41165: 0, 123087: 0, 90323: 0, 32983: 0, 57560: 1, 8409: 2, 16603: 2, 16610: 0, 16611: 3, 106726: 1, 24807: 3, 16618: 0, 235: 1, 16623: 0, 41201: 2, 24818: 1, 65779: 3, 250: 0, 24827: 2, 33020: 1, 73981: 1, 257: 1, 8450: 4, 24835: 